# Data Ingestion

In [ ]:
# !pip install -U langchain langchain-community langchain-core


  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 372.9 kB/s eta 0:00:06
   -------- ------------------------------- 0.5/2.5 MB 372.9 kB/s eta 0:00:06
   -------- ------------------------------- 0.5/2.5 MB 372.9 kB/s eta 0:00:06
   ------------ --------------------------- 0.8/2.5 MB 356.9 kB/s eta 0:00:05
   ------------ --------------------------- 0.8/2.5 MB 356.9

In [16]:
## document structure


from langchain_core.documents import Document 

In [17]:
doc=Document(
    page_content="This is the content of the document.",
    metadata={"source": "generated", "author": "AI"},
)

In [18]:
doc

Document(metadata={'source': 'generated', 'author': 'AI'}, page_content='This is the content of the document.')

In [19]:
## creating a simple txt document

import os
os.makedirs("../data/text_files", exist_ok=True)

In [20]:
sample_text = {
    
    "../data/text_files/doc1.txt": """This is the content of document , it is about python programming 
and its applications in data science.
Python is a versatile language that is widely used in various fields.
keyfeatures include simplicity, readability, and a vast ecosystem of libraries.
  
it has verstaile uses including web development, data analysis, machine learning, and automation.
  
    """,

}


for file_path, content in sample_text.items():   # using 2 variables, one for key and one for value
    with open(file_path, "w",encoding="utf-8") as f:
        f.write(content)
        
print("Sample text files created.")


Sample text files created.


In [29]:
## text loader

from langchain_community.document_loaders import TextLoader

loader=TextLoader("../data/text_files/doc1.txt", encoding="utf-8")  
document=loader.load()  # it is an object of the Textloader


In [ ]:
document  # automatically have the metadata and page_content

[Document(metadata={'source': '../data/text_files/doc1.txt'}, page_content='This is the content of document , it is about python programming \nand its applications in data science.\nPython is a versatile language that is widely used in various fields.\nkeyfeatures include simplicity, readability, and a vast ecosystem of libraries.\n  \nit has verstaile uses including web development, data analysis, machine learning, and automation.\n  \n    ')]

In [ ]:
## Directory Loader

